# Multi-class Classification of Anonymized Datasets

In [1]:
# The whole multi-class classification on target 'marital-status' on the adult dataset under several degrees of k-anonymization
# k = {(1), 3, 7, 11, 15, 19, 23, 27, 31, 35, 100}
# classifiers: Logistic Regression, Linear SVC, Random Forest, Gradient Boosting

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np

from sklearn import metrics, preprocessing, model_selection
from sklearn.ensemble import GradientBoostingClassifier as GradientBoosting, RandomForestClassifier as RandomForest
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

c:\users\tsarcevic\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
# Reading data
def read_data(filepath):
    #filepath = "../data/adult_all.csv"
    dataset = pd.read_csv(filepath, sep=r'\s*,\s*', na_values="*", engine='python', index_col=0)
    return dataset

In [4]:
# Reading anonymized data
def read_anon_data(filename):
    filepath = "../output/marital-status/"
    filepath += filename
    dataset = pd.read_csv(filepath, sep=r'\s*,\s*', na_values="*", engine='python', index_col=False)
    return dataset

In [5]:
# Preprocessing
def number_encode_features(ds):
    result = ds.copy()
    encoders = {}
    for feature in result.columns:
        if result.dtypes[feature] == np.object:
            encoders[feature] = preprocessing.LabelEncoder()
            result[feature] = encoders[feature].fit_transform(result[feature].astype(str))
    return result, encoders

In [6]:
# Scoring
def f1_micro(clf, X, y):
    # cross validation scores on number encoded data
    scores = model_selection.cross_val_score(clf, X, y, cv=10, scoring='f1_micro')
    print("F1 score (number encoded data): %0.2f (+/- %0.2f) %s"
          % (scores.mean(), scores.std() * 2, str(clf).split("(")[0]))
    return scores.mean()

In [7]:
# The pipeline
k_values = 1, 3, 11, 15, 19, 23, 27, 31, 35 #, 100 # 7 missing also

f1_scores_GB = []
f1_scores_LR = []
f1_scores_LSVC = []
f1_scores_RF = []

for k in k_values:
    # Read data
    if k==1:
        dataset = read_data("../data/adult_all.csv")
    else:
        dataset = read_anon_data("anonymized_equal_weights_k_" + str(k) + ".csv")
    
    # Preprocess
    dataset_encoded, encoders = number_encode_features(dataset)
    
    # define number-encoded features and target 'marital-status'
    y_ne = dataset_encoded['marital-status']
    X_ne = dataset_encoded.drop('marital-status', axis=1)
    
    # define binary features and target 'marital-status'
    y_bin = dataset['marital-status']
    X_bin = dataset.drop('marital-status', axis=1)
    X_bin = pd.get_dummies(X_bin)
    if k==1:
        del X_bin['sex_Male'], X_bin['income_>50K'], X_bin['native-country_Non-US']
        
    # define classifiers
    clf_GB = GradientBoosting(random_state=0)
    clf_RF = RandomForest(random_state=0)
    clf_LR = LogisticRegression(random_state=0)
    clf_LSVC = LinearSVC(random_state=0)
    
    print("k=" + str(k))
    # scoring
    f1_scores_GB.append(f1_micro(clf_GB, X_ne, y_ne))
    f1_scores_RF.append(f1_micro(clf_RF, X_ne, y_ne))
    if k==1:
        f1_scores_LR.append(f1_micro(clf_LR, X_bin, y_bin))
    else:
        f1_scores_LR.append(f1_micro(clf_LR, X_ne, y_ne))

k=1
F1 score (number encoded data): 0.85 (+/- 0.01) GradientBoostingClassifier
F1 score (number encoded data): 0.82 (+/- 0.01) RandomForestClassifier
F1 score (number encoded data): 0.84 (+/- 0.01) LogisticRegression
F1 score (number encoded data): 0.79 (+/- 0.07) LinearSVC
k=3
F1 score (number encoded data): 0.83 (+/- 0.02) GradientBoostingClassifier
F1 score (number encoded data): 0.80 (+/- 0.03) RandomForestClassifier
F1 score (number encoded data): 0.68 (+/- 0.02) LogisticRegression


MemoryError: 

In [ ]:
k_values = 1, 3, 7, 11, 15, 19, 23, 27, 31, 35 #, 100 

f1_scores_GB = []
f1_scores_LR = []
f1_scores_LSVC = []
f1_scores_RF = []

# (un)pickle results - instead of running the huge loop above, we can use results obtained in previous runs
for k in k_values:
    if k==1:
        filename = '../output/marital-status/classification-res/adult_multiclass_full'
    else:
        filename = '../output/marital-status/classification-res/adult_multiclass_k' + str(k)
    infile = open(filename,'rb')
    scores = pickle.load(infile)
    infile.close()
    
    f1_scores_GB.append(scores['Gradient Boosting'])
    f1_scores_LR.append(scores['Logistic Regression'])
    f1_scores_LSVC.append(scores['Linear SVC'])
    f1_scores_RF.append(scores['Random Forest'])

In [ ]:
# Visualize
def plot_results():
    fig = plt.figure(figsize=(14.56,9))
    ax = fig.add_subplot(2,2,1)
    ax.set_title('Logistic Regression')

    plt.plot(k_values, f1_scores_LR, marker='o')
    plt.ylabel('f1 score')
    plt.xlabel('k')

    ax = fig.add_subplot(2,2,2)
    ax.set_title('Linear SVC')

    plt.plot(k_values, f1_scores_LSVC, marker='o')
    plt.ylabel('f1 score')
    plt.xlabel('k')

    ax = fig.add_subplot(2,2,3)
    ax.set_title('Random Forest')

    plt.plot(k_values, f1_scores_RF, marker='o')
    plt.ylabel('f1 score')
    plt.xlabel('k')

    ax = fig.add_subplot(2,2,4)
    ax.set_title('Gradient Boosting')

    plt.plot(k_values, f1_scores_GB, marker='o')
    plt.ylabel('f1 score')
    plt.xlabel('k')

In [ ]:
plot_results()